In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide_310/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide_310/__init__.py
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv
/kaggle/input/amp-parkinsons-disease-progression-pre

In [2]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

import copy
import time

from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing

import matplotlib.pyplot as plt

from keras.layers import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM, GRU
from tensorflow import keras

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_281/985613018.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import LSTM
from keras.layers.core import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LSTM, GRU
from tensorflow import keras
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing

In [4]:
peptides = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv')
proteins = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv')
clinical = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')

In [ ]:
clinical

In [6]:
clinical['updrs_1'] = clinical['updrs_1'].fillna(method='ffill').fillna(method='bfill')
clinical['updrs_2'] = clinical['updrs_2'].fillna(method='ffill').fillna(method='bfill')
clinical['updrs_3'] = clinical['updrs_3'].fillna(method='ffill').fillna(method='bfill')
clinical['updrs_4'] = clinical['updrs_4'].fillna(method='ffill').fillna(method='bfill')
clinical['updrs_4'].fillna(0, inplace=True)

clinical.drop('upd23b_clinical_state_on_medication', axis=1, inplace=True)

clinical.isnull().sum()

visit_id       0
patient_id     0
visit_month    0
updrs_1        0
updrs_2        0
updrs_3        0
updrs_4        0
dtype: int64

In [ ]:
def cycle_check(all_merge):
    all_merge = all_merge.drop_duplicates().reset_index(drop=True)

    temp_cycle = []
    same_cycle = 1

    prev_patient_id = all_merge.loc[0, 'patient_id']
    prev_visit_month = all_merge.loc[0, 'visit_month']

    for i, j in all_merge.iterrows():
        if (prev_patient_id == j.patient_id) and (prev_visit_month == j.visit_month):
            temp_cycle.append(same_cycle)
        elif (prev_patient_id == j.patient_id) and (prev_visit_month != j.visit_month):
            same_cycle += 1
            temp_cycle.append(same_cycle)
        else:
            same_cycle = 1
            temp_cycle.append(same_cycle)

        prev_patient_id = j.patient_id
        prev_visit_month = j.visit_month

    return temp_cycle

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=999)
scaler = MinMaxScaler()

In [ ]:
def data_prep(Supclinical, peptide, protein, train=1):
    
    peptides = peptide[~peptide['UniProt'].isin(trash)].reset_index()
    proteins = protein[~protein['UniProt'].isin(trash)].reset_index()
    
    all_merge = Supclinical.merge(protein, how='left', on=['visit_id', 'visit_month', 'patient_id'])

    all_merge['UniProt'].fillna("other", inplace=True)
    all_merge['NPX'].fillna((all_merge['NPX'].mean()), inplace=True)

    if(train):
        all_merge[["UniProt"]] = enc.fit_transform(all_merge[["UniProt"]])
    else :
        all_merge[["UniProt"]] = enc.transform(all_merge[["UniProt"]])

        
    temp_cycle = cycle_check(all_merge)
    all_merge['cycle'] = temp_cycle
    
    if(train):
        cols_normalize = all_merge.columns.difference(['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id'])
        norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    else :
        all_merge['month_raw'] =  copy.deepcopy(all_merge.loc[:, 'visit_month'])
        cols_normalize = all_merge.columns.difference(['visit_id','patient_id','updrs_test','row_id','month_raw'])
        norm_train_df = pd.DataFrame(min_max_scaler.transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['visit_id','patient_id','updrs_test','row_id','month_raw']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    
    all_merge.drop(['index'],axis = 1, inplace = True)
    
    return all_merge

In [12]:
def data_prep(clinical, peptides, proteins, train=1):
    
    peptides = peptides[~peptides['UniProt'].isin(trash)].reset_index()
    proteins = proteins[~proteins['UniProt'].isin(trash)].reset_index()
    
    all_merge = clinical.merge(proteins, how='left', on=['visit_id', 'visit_month', 'patient_id'])

    all_merge['UniProt'].fillna("other", inplace=True)
    all_merge['NPX'].fillna((all_merge['NPX'].mean()), inplace=True)

    if(train):
        all_merge[["UniProt"]] = enc.fit_transform(all_merge[["UniProt"]])
    else :
        all_merge[["UniProt"]] = enc.transform(all_merge[["UniProt"]])

        
    temp_cycle = cycle_check(all_merge)
    all_merge['cycle'] = temp_cycle
    
    if(train):
        cols_normalize = all_merge.columns.difference(['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id'])
        norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    else :
        all_merge['month_raw'] =  copy.deepcopy(all_merge.loc[:, 'visit_month'])
        cols_normalize = all_merge.columns.difference(['visit_id','patient_id','updrs_test','row_id','month_raw'])
        norm_train_df = pd.DataFrame(min_max_scaler.transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['visit_id','patient_id','updrs_test','row_id','month_raw']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    
    all_merge.drop(['index'],axis = 1, inplace = True)
    
    return all_merge

In [9]:
trash = ['O00533','O14498','O15240','O15394','O43505','O60888','P00738','P01034','P01042','P01717',
         'P02452','P02649','P02751','P02753','P02787','P04075','P04156','P04180','P04216','P05060',
         'P05067','P05155','P05408','P05452','P06396','P07195','P07225','P07602','P07711','P07858',
         'P08133','P08253','P08571','P09104','P09486','P09871','P10645','P11142','P13521','P13591',
         'P13611','P13987','P14313','P14618','P17174','P19021','P23083','P23142','P39060','P40925',
         'P43121','P49908','P54289','P55290','P61278','P61769','P61916','P98160','Q02818','Q06481',
         'Q08380','Q12907','Q13332','Q14118','Q14508','Q14515','Q15904','Q16610','Q6UXB8','Q7Z3B1',
         'Q8NBJ4','Q92520','Q92823','Q96KN2','Q99435','Q99674','Q9BY67','Q9NQ79','Q9NYU2','Q9UHG2',
         'P01594','Q13449','Q99829']

In [11]:
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

def data_prep(clinical, peptides, proteins, train=1):
    
    peptides = peptides[~peptides['UniProt'].isin(trash)].reset_index()
    proteins = proteins[~proteins['UniProt'].isin(trash)].reset_index()
    
    all_merge = clinical.merge(proteins, how='left', on=['visit_id', 'visit_month', 'patient_id'])

    all_merge['UniProt'].fillna("other", inplace=True)
    all_merge['NPX'].fillna((all_merge['NPX'].mean()), inplace=True)

    if(train):
        enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=999)
        all_merge[["UniProt"]] = enc.fit_transform(all_merge[["UniProt"]])
    else :
        all_merge[["UniProt"]] = enc.transform(all_merge[["UniProt"]])

        
    temp_cycle = cycle_check(all_merge)
    all_merge['cycle'] = temp_cycle
    
    scaler = MinMaxScaler()
    
    if(train):
        cols_normalize = all_merge.columns.difference(['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id'])
        norm_train_df = pd.DataFrame(scaler.fit_transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    else :
        all_merge['month_raw'] =  copy.deepcopy(all_merge.loc[:, 'visit_month'])
        cols_normalize = all_merge.columns.difference(['visit_id','patient_id','updrs_test','row_id','month_raw'])
        norm_train_df = pd.DataFrame(scaler.transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['visit_id','patient_id','updrs_test','row_id','month_raw']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    
    all_merge.drop(['index'],axis = 1, inplace = True)
    
    return all_merge

In [13]:
data_prep_train = data_prep(clinical, peptides, proteins, train=1)
data_prep_train

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,UniProt,NPX,cycle
0,55_0,55,0.000000,10.0,6.0,15.0,0.0,0.000000,0.000018,0.0000
1,55_0,55,0.000000,10.0,6.0,15.0,0.0,0.006897,0.000064,0.0000
2,55_0,55,0.000000,10.0,6.0,15.0,0.0,0.013793,0.000051,0.0000
3,55_0,55,0.000000,10.0,6.0,15.0,0.0,0.020690,0.000007,0.0000
4,55_0,55,0.000000,10.0,6.0,15.0,0.0,0.027586,0.000161,0.0000
...,...,...,...,...,...,...,...,...,...,...
142724,65043_48,65043,0.444444,7.0,6.0,13.0,0.0,0.993103,0.000064,0.6250
142725,65043_54,65043,0.500000,4.0,8.0,11.0,1.0,1.000000,0.005936,0.6875
142726,65043_60,65043,0.555556,6.0,6.0,16.0,1.0,1.000000,0.005936,0.7500
142727,65043_72,65043,0.666667,3.0,9.0,14.0,1.0,1.000000,0.005936,0.8125


In [14]:
# test and submission.

submission = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv")

test_proteins = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_proteins.csv")
test_proteins.drop(['group_key'],axis = 1, inplace = True)

test_peptides = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test_peptides.csv")
test_peptides.drop(['group_key'],axis = 1, inplace = True)

test_clinical = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/test.csv")
test_clinical.drop(['group_key'],axis = 1, inplace = True)

In [15]:
data_prep_test = data_prep(test_clinical, test_peptides, test_proteins, train=0)
data_prep_test

,visit_id,visit_month,patient_id,updrs_test,row_id,UniProt,NPX,cycle,month_raw
0,3342_0,0.000000,3342,updrs_1,3342_0_updrs_1,1.000000,0.006240,0.0,0
1,3342_0,0.000000,3342,updrs_2,3342_0_updrs_2,1.000000,0.006240,0.0,0
2,3342_0,0.000000,3342,updrs_3,3342_0_updrs_3,1.000000,0.006240,0.0,0
3,3342_0,0.000000,3342,updrs_4,3342_0_updrs_4,1.000000,0.006240,0.0,0
4,50423_0,0.000000,50423,updrs_1,50423_0_updrs_1,0.000000,0.000054,0.0,0
...,...,...,...,...,...,...,...,...,...
1187,3342_6,0.055556,3342,updrs_4,3342_6_updrs_4,0.993103,0.000024,0.0,6
1188,50423_6,0.055556,50423,updrs_1,50423_6_updrs_1,1.000000,0.006240,0.0,6
1189,50423_6,0.055556,50423,updrs_2,50423_6_updrs_2,1.000000,0.006240,0.0,6
1190,50423_6,0.055556,50423,updrs_3,50423_6_updrs_3,1.000000,0.006240,0.0,6


In [ ]:
def data_prep(clinical, peptides, proteins, train=1):
    
    peptides = peptides[~peptides['UniProt'].isin(trash)].reset_index()
    proteins = proteins[~proteins['UniProt'].isin(trash)].reset_index()
    
    all_merge = clinical.merge(proteins, how='left', on=['visit_id', 'visit_month', 'patient_id'])

    all_merge['UniProt'].fillna("other", inplace=True)
    all_merge['NPX'].fillna((all_merge['NPX'].mean()), inplace=True)

    if(train):
        enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=999)
        all_merge[["UniProt"]] = enc.fit_transform(all_merge[["UniProt"]])
    else:
        all_merge["UniProt"] = enc.transform(all_merge[["UniProt"]])

    temp_cycle = cycle_check(all_merge)
    all_merge['cycle'] = temp_cycle
    
    if(train):
        cols_normalize = all_merge.columns.difference(['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id'])
        norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['updrs_1','updrs_2','updrs_3','updrs_4','visit_id','patient_id']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    else :
        all_merge['month_raw'] =  copy.deepcopy(all_merge.loc[:, 'visit_month'])
        cols_normalize = all_merge.columns.difference(['visit_id','patient_id','updrs_test','row_id','month_raw'])
        norm_train_df = pd.DataFrame(min_max_scaler.transform(all_merge[cols_normalize]), 
                             columns=cols_normalize, 
                             index=all_merge.index)
        join_df = all_merge[['visit_id','patient_id','updrs_test','row_id','month_raw']].join(norm_train_df)
        all_merge = join_df.reindex(columns = all_merge.columns)
    
    all_merge.drop(['index'],axis = 1, inplace = True)
    
    return all_merge

In [ ]:
data_prep_test = data_prep(test_clinical, test_peptides, test_proteins, train=0)
data_prep_test

In [16]:
sequence_length = 50
def sequence_generator(feature_df, seq_length, seq_cols):
    if(feature_df.shape[0] < sequence_length):
        feature_df = pd.concat([feature_df, pd.concat([feature_df[-1:]]*(sequence_length+1 - feature_df.shape[0]))])
    feature_array = feature_df[seq_cols].values
    num_elements = feature_array.shape[0]    
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield feature_array[start:stop, :]

In [17]:
def label_generator(label_df, seq_length, label):
    if(label_df.shape[0] < sequence_length):
        label_df = pd.concat([label_df, pd.concat([label_df[-1:]]*(sequence_length+1 - label_df.shape[0]))])
    label_array = label_df[label].values
    num_elements = label_array.shape[0]    
    return label_array[seq_length:num_elements, :]

In [18]:
seq_cols = ['visit_month','UniProt','NPX','cycle']

In [19]:
def train_sed_gen(data_prep_train):
    seq_gen_train = (list(sequence_generator(data_prep_train[data_prep_train['patient_id'] == patient_id], sequence_length, seq_cols))
           for patient_id in data_prep_train['patient_id'].unique())

    #print(list(seq_gen))
    seq_gen_train = [ele for ele in seq_gen_train if ele != []]

    seq_gen_train_f = np.concatenate(list(seq_gen_train)).astype(np.float32)
    return seq_gen_train_f

In [20]:
seq_gen_train_f = train_sed_gen(data_prep_train)
seq_gen_train_f.shape

(130329, 50, 4)

In [21]:
def train_label_gen(data_prep_train, target):
    label_gen = [label_generator(data_prep_train[data_prep_train['patient_id'] == patient_id], sequence_length, [target]) 
             for patient_id in data_prep_train['patient_id'].unique()]
    label_set = np.concatenate(label_gen).astype(np.float32)
    return label_set

In [22]:
target_updrs_1 = train_label_gen(data_prep_train, 'updrs_1')
print(target_updrs_1.shape)
target_updrs_2 = train_label_gen(data_prep_train, 'updrs_2')
print(target_updrs_2.shape)
target_updrs_3 = train_label_gen(data_prep_train, 'updrs_3')
print(target_updrs_3.shape)
target_updrs_4 = train_label_gen(data_prep_train, 'updrs_4')
print(target_updrs_4.shape)

(130329, 1)
(130329, 1)
(130329, 1)
(130329, 1)


In [23]:
data_prep_test['row_id'].unique()

array(['3342_0_updrs_1', '3342_0_updrs_2', '3342_0_updrs_3',
       '3342_0_updrs_4', '50423_0_updrs_1', '50423_0_updrs_2',
       '50423_0_updrs_3', '50423_0_updrs_4', '3342_6_updrs_1',
       '3342_6_updrs_2', '3342_6_updrs_3', '3342_6_updrs_4',
       '50423_6_updrs_1', '50423_6_updrs_2', '50423_6_updrs_3',
       '50423_6_updrs_4'], dtype=object)

In [24]:
# test seq
def test_seq_gen(data_prep_test):
    seq_gen_test = (list(sequence_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, seq_cols))
           for row_id in data_prep_test['row_id'].unique())

    #print(list(seq_gen))
    seq_gen_test = [ele for ele in seq_gen_test if ele != []]

    seq_gen_test_f = np.concatenate(list(seq_gen_test)).astype(np.float32)
    return seq_gen_test_f

In [25]:
seq_gen_test_f = test_seq_gen(data_prep_test)
seq_gen_test_f.shape

(792, 50, 4)

In [26]:
def test_data_save(data_prep_test):
    row_id_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['row_id']) 
             for row_id in data_prep_test['row_id'].unique()]
    row_id_out = np.concatenate(row_id_out)
    visit_month_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['month_raw']) 
             for row_id in data_prep_test['row_id'].unique()]
    visit_month_out = np.concatenate(visit_month_out)
    visit_id_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['visit_id']) 
             for row_id in data_prep_test['row_id'].unique()]
    visit_id_out = np.concatenate(visit_id_out)
    updrs_test_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['updrs_test']) 
             for row_id in data_prep_test['row_id'].unique()]
    updrs_test_out = np.concatenate(updrs_test_out)
    
    patient_id_out = [label_generator(data_prep_test[data_prep_test['row_id'] == row_id], sequence_length, ['patient_id']) 
             for row_id in data_prep_test['row_id'].unique()]
    patient_id_out = np.concatenate(patient_id_out)


    row_id_out_df = pd.DataFrame(row_id_out, columns = ['row_id'])
    visit_month_out_df = pd.DataFrame(visit_month_out, columns = ['visit_month'])
    visit_id_out_df = pd.DataFrame(visit_id_out, columns = ['visit_id'])
    updrs_test_out_df = pd.DataFrame(updrs_test_out, columns = ['updrs_test'])
    patient_id_out_df = pd.DataFrame(patient_id_out, columns = ['patient_id'])
    pred_df = pd.concat([visit_id_out_df, visit_month_out_df,patient_id_out_df,updrs_test_out_df,row_id_out_df], axis=1)
    return pred_df

In [27]:
pred_df = test_data_save(data_prep_test)
print(pred_df.shape)
pred_df

(792, 5)


,visit_id,visit_month,patient_id,updrs_test,row_id
0,3342_0,0,3342,updrs_1,3342_0_updrs_1
1,3342_0,0,3342,updrs_2,3342_0_updrs_2
2,3342_0,0,3342,updrs_3,3342_0_updrs_3
3,3342_0,0,3342,updrs_4,3342_0_updrs_4
4,50423_0,0,50423,updrs_1,50423_0_updrs_1
...,...,...,...,...,...
787,3342_6,6,3342,updrs_4,3342_6_updrs_4
788,50423_6,6,50423,updrs_1,50423_6_updrs_1
789,50423_6,6,50423,updrs_2,50423_6_updrs_2
790,50423_6,6,50423,updrs_3,50423_6_updrs_3


In [28]:
pred_df.groupby(['row_id', 'visit_month','visit_id'])['row_id', 'visit_month','visit_id'].size()

/tmp/ipykernel_281/1665520064.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  pred_df.groupby(['row_id', 'visit_month','visit_id'])['row_id', 'visit_month','visit_id'].size()


row_id           visit_month  visit_id
3342_0_updrs_1   0            3342_0       1
3342_0_updrs_2   0            3342_0       1
3342_0_updrs_3   0            3342_0       1
3342_0_updrs_4   0            3342_0       1
3342_6_updrs_1   6            3342_6      98
3342_6_updrs_2   6            3342_6      98
3342_6_updrs_3   6            3342_6      98
3342_6_updrs_4   6            3342_6      98
50423_0_updrs_1  0            50423_0     98
50423_0_updrs_2  0            50423_0     98
50423_0_updrs_3  0            50423_0     98
50423_0_updrs_4  0            50423_0     98
50423_6_updrs_1  6            50423_6      1
50423_6_updrs_2  6            50423_6      1
50423_6_updrs_3  6            50423_6      1
50423_6_updrs_4  6            50423_6      1
dtype: int64

In [30]:
import tensorflow.keras.backend as K
def smape_loss(y_true, y_pred):
    epsilon = 0.1
    summ = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = K.abs(y_pred - y_true) / summ * 2.0
    return smape

In [31]:
def plot_model_accuracy(model_updrs_1_name_history, width = 10, height = 10):
  fig_acc = plt.figure(figsize=(width, height))
  plt.plot(model_updrs_1_name_history.history['mean_absolute_error'])
  plt.plot(model_updrs_1_name_history.history['val_mean_absolute_error'])
  plt.title('model')
  plt.ylabel('mse')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

In [32]:
import time
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Conv1D, MaxPooling1D
from keras import callbacks
from keras.callbacks import History

# define a dictionary with information about each model
models_dict = {
    'updrs_1': {'target': target_updrs_1, 'model_name': 'updrs_1_binary_model_updrs_1.h5'},
    'updrs_2': {'target': target_updrs_2, 'model_name': 'updrs_2_binary_model_updrs_2.h5'},
    'updrs_3': {'target': target_updrs_3, 'model_name': 'updrs_3_binary_model_updrs_3.h5'},
    'updrs_4': {'target': target_updrs_4, 'model_name': 'updrs_4_binary_model_updrs_4.h5'}
}

In [ ]:
test_predicts = []

for model_name, model_info in models_dict.items():
    print(f'working on {model_name}')
    
    # get target and model name for this model
    target = model_info['target']
    model_name = model_info['model_name']

    # get output dimension
    out_dim = target.shape[1]

    # create model
    model = Sequential()
    model.add(Bidirectional(LSTM(units=100, input_shape=(sequence_length, seq_gen_train_f.shape[2]), return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Bidirectional(LSTM(units=50, return_sequences=False)))
    model.add(Dropout(0.2))
    model.add(Dense(out_dim))
    model.add(Activation("linear"))
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    
    # define callbacks for early stopping and model checkpoint
    early_stop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='min')
    model_checkpoint = callbacks.ModelCheckpoint(model_name, monitor='val_loss', save_best_only=True, mode='min', verbose=0)
    
    # train model
    epochs = 10
    batch_size = 100
    num_batches = seq_gen_train_f.shape[0] // batch_size
    with tqdm(total=epochs * num_batches, desc=f"{model_name}") as pbar:
        for i in range(epochs):
            for j in range(num_batches):
                start_idx = j * batch_size
                end_idx = (j + 1) * batch_size
                x_batch = seq_gen_train_f[start_idx:end_idx]
                y_batch = target[start_idx:end_idx]
                history = model.train_on_batch(x_batch, y_batch)
                pbar.update(1)
                
    history = History()  
    model.fit(seq_gen_train_f, target, epochs=30, batch_size=100, validation_split=0.05, verbose=2, callbacks=[early_stop, model_checkpoint, history])  # callbacks 인수에 History 객체 추가
    train_predict = model.predict(seq_gen_train_f)
    test_predict = model.predict(seq_gen_test_f)
    test_predicts.append(test_predict)
    plot_model_accuracy(history, 5, 5)

    # save trained model
    model.save(model_name)

    # print results
    print(f"Train predict shape: {train_predict.shape}")
    print(train_predict)
    print(f"Test predict shape: {test_predict.shape}")
    print(test_predict)

working on updrs_1


updrs_1_binary_model_updrs_1.h5:  62%|██████▏   | 8068/13030 [18:59<11:22,  7.27it/s]

Reference: https://www.kaggle.com/code/joonyoungjang/amp-with-bi-lstm-conv/notebook